In [ ]:
!pip install pydantic langchain-teddynote langchain_community langchain_huggingface langchain_openai

In [ ]:
from google.colab import output

output.clear()

In [ ]:
import os

from google.colab import userdata

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_PROJECT"] = "LangChain 실습"
# Langchain API key 입력
# os.environ["LANGCHAIN_API_KEY"] = userdata.get('langsmith_api_key')

In [ ]:
import zipfile

# 데이터 처리 및 분석
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm
import seaborn as sns
from matplotlib import pyplot as plt
import datetime

from sklearn.metrics.pairwise import cosine_similarity

# 모델 저장
import joblib

# To ignore all warnings
import warnings
warnings.filterwarnings('ignore')



# 데이터 불러오기

In [ ]:
bl_data_sampled=pd.read_csv('bl_sample_data.csv')

display(bl_data_sampled.head())

,HSCD,SUPLY_CO_NAME,PRDT_DC_VAL
0,330499,"MESOESTETIC,. S.L.",BODYSHOCK LOCAL REDUCER 200ML (FACE CREAM)
1,330499,"MESOESTETIC,. S.L.",COSMELAN 2 (FACE CREAM)
2,330499,"MESOESTETIC,. S.L.",ANTI STRESS FACE MASK 100ML (FACE CREAM)
3,330499,KRYOLAN GMBH CHEMISCHE FABRIK,BLOT POWDER
4,330499,KRYOLAN GMBH CHEMISCHE FABRIK,DERMACOLOR CAMOUFLAGE FLUID


In [ ]:
bk_data_sampled=pd.read_excel('BK_330499_수정.xlsx')

display(bk_data_sampled.head())

,NO,PID,CORPNAME,PRD_NAME,PRD,URL
0,1,3723376,"CELLVIO COSMETIC CO.,LTD.",Doldori Premium Gold Essence,Our patented mask pack redefines skincare with...,https://buykorea.org/ec/prd/selectGoodsDetail....
1,2,3722918,Brade,Exosome Exo-V Skin Boos,The EXO-V Exosome Skin Booster is used in Kore...,https://buykorea.org/ec/prd/selectGoodsDetail....
2,3,3722956,"BIOVAIKOREA Co.,LTD",Oil-Free Ultra-Moisturizing Lotion with Birch Sap,DETAILS\nBENEFITS\n\n\n• Soothing irritated sk...,https://buykorea.org/ec/prd/selectGoodsDetail....
3,4,3722734,Hit Company,CHARNE - Shiny Brightening Cream,Charne Shiny Brightening Cream\n\nVarious natu...,https://buykorea.org/ec/prd/selectGoodsDetail....
4,5,3717513,"Wooanjoo Co.,Ltd",DEAR OHNEUL Singreen Donkey Toner Pad,Product Features:\n\n－ Hypoallergenic: Dermato...,https://buykorea.org/ec/prd/selectGoodsDetail....


# Langsmith , LLM API KEY 설정

In [ ]:
from langchain_teddynote import logging

# logging.langsmith("KOTRA_BL_data_matching")

In [ ]:
from langchain_community.chat_models import ChatPerplexity
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate

from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


os.environ["OPENAI_API_KEY"] = userdata.get('KOTRA2_openai_api_key')


# API 사용 rate 제한 설정


In [ ]:
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter_openai = InMemoryRateLimiter(
    requests_per_second=5,  # maximum query per day - 10000 (tier 1)
    check_every_n_seconds=0.1,  # Wake up every 100 ms to check whether allowed to make a request,
    max_bucket_size=10,  # Controls the maximum burst size.
    )

<ipython-input-9-ba5702665662>:3: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter_perplexity = InMemoryRateLimiter(


# 사용 모델 설정

In [ ]:
chat_openai = ChatOpenAI(temperature=0,  model="gpt-4o-mini", rate_limiter=rate_limiter_openai)

# OpenAI API 이용 결과 (model : chatgpt 4o-mini)

# buyKOREA 설명 처리 프롬프트문

---------------------

+ 제품 설명 분석
+ 20개의 keyword를 부사, 형용사 없이 나열한 하나의 문장으로 출력
+ 제품의 용도를 잘 설명하는 20개의 한 단어 키워드 추출 (**중복 없이**)
+ 영어로 작성 (토큰 수 줄이기 + llm이 잘 인식하도록)
+ **키워드는 절이 아니라 단어여야 함 (다시 강조 - 절로 추출 시 필요없는 부사, 전치사 다수 포함)**  
+ **제품의 용량 관련된 키워드는 제외**

In [ ]:
class DescriptionSummary(BaseModel):
    type : str = Field(description="product type")
    purpose : str = Field(description="product purpose")
    brand : str = Field(description='product brand')
    summary : str = Field(description="summary of description")

parser= PydanticOutputParser(pydantic_object=DescriptionSummary)

In [ ]:
prompt_buykorea = PromptTemplate.from_template(
    """You are given data about product's description(PRDT_DESC), written by manufacturing company(PRDT_COM).
    All products fall under HS Code 330499, which is a subcategory of 3304.
    The description for 3304 is as follows:
    33.04 - Beauty or make-up preparations and preparations for the care of the skin (other than medicaments), including sunscreen or sun tan preparations; manicure or pedicure preparations.

    Using this information, complete the following tasks:
    Provide a simple summary for description.

    Example :
    PRDT_DESC : 'FIORESE Pure Hydra Water Cream contains Centella Asiatica and Hyaluronic Acid, effectively delivering hydration and providing instant soothing benefits.
    With Niacinamide and Adenosine, it is a dual-functional cosmetic for brightening and wrinkle care. This cream helps create healthy, smooth, and radiant skin.
    The lightweight, hydrating formula absorbs quickly, delivering moisture deep into the skin with a fresh, non-sticky finish. Suitable for all skin types, especially sensitive, dull, and uneven skin.

    For Sensitive Skin: Gentle, irritation-free formula, suitable even for sensitive skin.

    FIORESE Brand: Fiorese is dedicated to clean beauty, using ingredients sourced from nature. 100% cruelty-free.'

    PRDT_COM : DAMOA COSMETICS.CO.LTD

    Output Example : The FIORESE Pure Hydra Water Cream is a lightweight, hydrating cream containing Centella Asiatica and Hyaluronic Acid, providing deep moisture and soothing benefits.
    It also includes Niacinamide and Adenosine, making it a dual-functional cosmetic for brightening and wrinkle care.
    The formula absorbs quickly, leaving a fresh, non-sticky finish, and is suitable for all skin types, especially sensitive, dull, and uneven skin.
    The product is gentle and irritation-free, ideal for sensitive skin. Fiorese is committed to clean beauty with cruelty-free, nature-sourced ingredients.

    PRODUCT DESCRIPTION(PRDT_DESC) : {prdt_desc}
    MANUFACTURING COMPANY(PRDT_COM) : {prdt_com}
    """
)


chain_buykorea = type_prompt_buykorea | chat_openai

In [ ]:
buykorea_preprocessed_dict_openai={}

for index, row in bk_data_sampled.iterrows():
  response_buykorea = chain_buykorea.invoke({"input": row['PRD']})
  print(response_buykorea)
  buykorea_preprocessed_dict_openai[row['PRD_NAME']]=response_buykorea

type='mask pack' purpose='skincare treatment' brand='not specified' summary='Mask pack offers new application method. Product available for export and sold in Korea. Manufacturing process managed directly. Essence formulation and mask sheet development handled in-house. Product designed for hydration, brightening, revitalization.'
type='skin booster' purpose='improves skin regeneration, elasticity, wrinkle reduction, moisture retention' brand='EXO-V' summary='used in clinics, home care, enhances scalp nutrients, strengthens hair roots'
type='lotion' purpose='moisturizing skin' brand='not specified' summary='Soothing skin, restoring moisture, penetrating skin, calming irritation.'
type='cream' purpose='whitening, wrinkle improvement' brand='Charne' summary='Natural ingredients support skin health, nutrition, vitality, firmness, elasticity.'
type='skincare pads' purpose='cleanse pores, hydrate skin, control sebum' brand='not specified' summary='Hypoallergenic pads improve pores, provide 

In [ ]:
buykorea_preprocessed_dict_openai = {
    k: {'type': v.type, 'purpose': v.purpose, 'brand': v.brand, 'summary': v.summary}
    for k, v in buykorea_preprocessed_dict_openai.items()
}

In [ ]:
buykorea_preprocessed_openai=pd.DataFrame.from_dict(buykorea_preprocessed_dict_openai, orient='index', columns=['type', 'purpose', 'brand', 'summary'])
buykorea_preprocessed_openai.reset_index(inplace=True)
buykorea_preprocessed_openai.rename(columns={'index':'product_name'}, inplace=True)
buykorea_preprocessed_openai.to_csv('buykorea_preprocessed_openai.csv', index=False)

display(buykorea_preprocessed_openai.head())

,product_name,type,purpose,brand,summary
0,Doldori Premium Gold Essence,mask pack,skincare treatment,not specified,Mask pack offers new application method. Produ...
1,Exosome Exo-V Skin Boos,skin booster,"improves skin regeneration, elasticity, wrinkl...",EXO-V,"used in clinics, home care, enhances scalp nut..."
2,Oil-Free Ultra-Moisturizing Lotion with Birch Sap,lotion,moisturizing skin,not specified,"Soothing skin, restoring moisture, penetrating..."
3,CHARNE - Shiny Brightening Cream,cream,"whitening, wrinkle improvement",Charne,"Natural ingredients support skin health, nutri..."
4,DEAR OHNEUL Singreen Donkey Toner Pad,skincare pads,"cleanse pores, hydrate skin, control sebum",not specified,"Hypoallergenic pads improve pores, provide cle..."


# BL 데이터 이름 처리 프롬프트문

----------------------------

+ 업무 배경 설명 (수입자, 공급자, 상품 설명이 존재함을 인식)
+ HSCODE 4자리 설명을 추가함 (6자리보다 더 광범위한 개념 인식)
+ 공급자와 상품 이름 정보에 기반한 제품 설명 생성
+ 인터넷 검색으로 추가 정보 검색 지시
+ 의미가 확실치 않은 경우 생성 제한
+ 지시 내용별 예시 제공 (one-shot)

In [ ]:
prompt_bldata = PromptTemplate.from_template(
    """You are given data where IMP_CO_NAME imports products (PRDT_DC_VAL) from SUPLY_CO_NAME.
    All products fall under HS Code 330499, which is a subcategory of 3304.
    The description for 3304 is as follows:
    33.04 - Beauty or make-up preparations and preparations for the care of the skin (other than medicaments), including sunscreen or sun tan preparations; manicure or pedicure preparations.

    Using this information, complete the following tasks:
    Provide a simple explanation for PRDT_DC_VAL using both SUPLY_CO_NAME and PRDT_DC_VAL.
    Use publicly available product descriptions from websites to help.
    If the meaning is unclear, do not force an explanation.

    Example:
    PRDT_DC_VAL: COSMELAN 2 (FACE CREAM)
    SUPLY_CO_NAME: MESOESTETIC, S.L.
    Output Example: This product is called COSMELAN 2, a face cream designed to reduce pigmentation and improve skin tone. It is often used for treating melasma and other skin discolorations by inhibiting melanin production.
    Extract up to 5 key Korean keywords that describe the product's main features.
    Output Example: pigmentation, skin tone, melasma, melanin, face cream
    Identify the type of product.
    Output Example: face cream
    Extract only the product name from PRDT_DC_VAL, excluding any volume, symbols, special characters, or colors.
    Output Example: COSMELAN 2

    PRODUCT DESCRIPTION(PRDT_DC_VAL) : {prdt_desc}
    SUPPLEYER NAME(SUPLY_CO_NAME) : {suply_name}
    """
)


chain_bldata = type_prompt_bldata | chat_openai

In [ ]:
bldata_preprocessed_dict_openai={}

for index, row in tqdm(bl_data_sampled.iterrows()):
  response_bldata = chain_bldata.invoke({"input": row['PRDT_DC_VAL']})
  if index%100==0:
    print(f"Product Name : {row['PRDT_DC_VAL']} \n")
    print(response_bldata, '\n')
  bldata_preprocessed_dict_openai[row['PRDT_DC_VAL']]=response_bldata

1it [00:01,  1.06s/it]

Product Name : BODYSHOCK LOCAL REDUCER 200ML (FACE CREAM) 

type='Face Cream' purpose='Reduces local fat and enhances skin appearance' brand='Bodyshock' summary='200ml cream with essential oils for targeted facial care.' 



101it [02:05,  1.26s/it]

Product Name : AVEENO BABY DERMEXA MOISTURING CREAM 206G(1 PAC X 12 PCS) 

type='Moisturizing Cream' purpose='Hydrate and soothe baby skin' brand='Aveeno Baby' summary='Aveeno Baby Dermexa Moisturizing Cream provides essential hydration and relief for dry, sensitive skin.' 



201it [04:26,  1.68s/it]

...

2801it [1:00:23,  1.43s/it]

Product Name : SWISS IMAGE BRAND - SI EN SOOTH.CLEAN.MILK 200ML 

type='Beauty preparation' purpose='Cleansing and soothing skin' brand='Swiss Image' summary='SI En Sooth.Clean.Milk is a gentle milk cleanser that soothes and purifies the skin.' 



2901it [1:02:20,  1.25s/it]

Product Name : FACE CARE PRODUCTS OTHER THAN FACE MASK - NS00903 NASHI ARGAN - THE BALM - MOISTURIZING AFTER SHAVE, 100 ML 

type='Face Care Product' purpose='Moisturizing after shave' brand='Nashi Argan' summary='Moisturizing balm for post-shave care, enriched with argan oil for hydration.' 



2932it [1:03:02,  1.29s/it]


In [ ]:
bldata_preprocessed_dict_openai = {
    k: {'type': v.type, 'purpose': v.purpose, 'brand': v.brand, 'summary': v.summary}
    for k, v in bldata_preprocessed_dict_openai.items()
}

AttributeError: 'dict' object has no attribute 'type'

In [ ]:
bldata_preprocessed_openai=pd.DataFrame.from_dict(bldata_preprocessed_dict_openai, orient='index', columns=['type', 'purpose', 'brand', 'summary'])
bldata_preprocessed_openai.reset_index(inplace=True)
bldata_preprocessed_openai.rename(columns={'index':'product_name'}, inplace=True)
bldata_preprocessed_openai.to_csv('bldata_preprocessed_openai.csv', index=False)

display(bldata_preprocessed_openai.head())

,product_name,type,purpose,brand,summary
0,BODYSHOCK LOCAL REDUCER 200ML (FACE CREAM),Face Cream,Reduces local fat and enhances skin appearance,Bodyshock,200ml cream with essential oils for targeted f...
1,COSMELAN 2 (FACE CREAM),Face Cream,Skin care and treatment,Cosmelan,Cosmelan 2 is a face cream designed to reduce ...
2,ANTI STRESS FACE MASK 100ML (FACE CREAM),Face Cream,Reduces stress and promotes relaxation for the...,Anti Stress,A soothing face mask enriched with essential o...
3,BLOT POWDER,Beauty Powder,Enhance skin appearance and texture,N/A,"BLOT POWDER absorbs excess oil, providing a ma..."
4,DERMACOLOR CAMOUFLAGE FLUID,Beauty or make-up preparation,Camouflage skin imperfections,Dermacolor,Dermacolor Camouflage Fluid provides effective...


In [ ]:
bldata_preprocessed_openai

,product_name,type,purpose,brand,summary
0,BODYSHOCK LOCAL REDUCER 200ML (FACE CREAM),Face Cream,Reduces local fat and enhances skin appearance,Bodyshock,200ml cream with essential oils for targeted f...
1,COSMELAN 2 (FACE CREAM),Face Cream,Skin care and treatment,Cosmelan,Cosmelan 2 is a face cream designed to reduce ...
2,ANTI STRESS FACE MASK 100ML (FACE CREAM),Face Cream,Reduces stress and promotes relaxation for the...,Anti Stress,A soothing face mask enriched with essential o...
3,BLOT POWDER,Beauty Powder,Enhance skin appearance and texture,N/A,"BLOT POWDER absorbs excess oil, providing a ma..."
4,DERMACOLOR CAMOUFLAGE FLUID,Beauty or make-up preparation,Camouflage skin imperfections,Dermacolor,Dermacolor Camouflage Fluid provides effective...
...,...,...,...,...,...
2796,P-GINKGO LEAVES EXTRACT (COSMETIC PRODUCTS FOR...,Cosmetic Extract,Research and Development,P-Ginkgo,Ginkgo leaves extract for cosmetic application...
2797,BAMBOO CHARCOAL BLOTTING TISSUE GRAY 160 PCS (...,Beauty Product,Blotting excess oil and shine from skin,N/A,Bamboo charcoal blotting tissue effectively ab...
2798,FINISHED PRODUCTS- KITS LYSOFIX (NCV),Beauty Kit,Skin care and fragrance,Lysifix,"Includes essential oils, perfumes, and beauty ..."
2799,FINISHED PRODUCTS-COLOR ETHICS (NCV),Cosmetic Products,Enhance beauty and personal care,Color Ethics,"Includes essential oils, perfumes, beauty prep..."


# 수출한 기업은 다르지만, 수출한 품목의 이름은 같은 경우가 있어 불일치 발생
(중복 제거됨)

In [ ]:
# 같은 품목 중 다른 기업이 수출한 이력이 있는 경우, 다른 수출건으로 집계됨

bl_data_sampled.shape

(2932, 3)

In [ ]:
# 제품 이름에 기반해, 제품 수입 확률이 높은 해외 바이어 매칭이므로
# 중복되는 제품 이름은 하나의 키워드 및 임베딩으로 통일

len(bldata_preprocessed_dict_openai)

2801

# 임베딩

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

# 오픈소스 임베딩 모델 다운로드
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def text_embedding(text):
  embedding_text=embeddings.embed_query(text)
  return np.array(embedding_text).reshape(1,-1)

In [ ]:
tqdm.pandas()

# 임베딩할 텍스트

buykorea_preprocessed_openai=pd.read_csv('./buykorea_preprocessed_openai.csv')
bldata_preprocessed_openai=pd.read_csv('./bldata_preprocessed_openai_241119.csv')

buykorea_preprocessed_openai['buykorea_preprocessed']=buykorea_preprocessed_openai['type']+":"+buykorea_preprocessed_openai['purpose']
bldata_preprocessed_openai['bldata_preprocessed']=bldata_preprocessed_openai['type']+":"+bldata_preprocessed_openai['purpose']
# 키워드로 구성된 문장을 임베딩
buykorea_preprocessed_openai['buykorea_embedding']=buykorea_preprocessed_openai['buykorea_preprocessed'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_embedding']=bldata_preprocessed_openai['bldata_preprocessed'].progress_apply(text_embedding)

bldata_sample_len=bldata_preprocessed_openai.shape[0]

100%|██████████| 2801/2801 [05:21<00:00,  8.71it/s]


In [ ]:
openai_similarity_df=pd.DataFrame(columns=['buykorea_product_name', 'buykorea_keyword', 'bldata_product_name', 'bldata_keyword', 'cosine_similarity'])

for index, row in buykorea_preprocessed_openai.iterrows():

  # 각 buykorea 상품 (10개) 에,
  # 2800개의 수출 기록 존재하는 제품 이름과의 유사도 계산
  # buykorea 상품별로 2800개의 유사도가 계산되어 저장된 데이터프레임 생성

  similarity_df=pd.DataFrame({'buykorea_product_name':[row['product_name']]*bldata_sample_len,
                               'buykorea_keyword': [row['buykorea_preprocessed']]*bldata_sample_len,
                               'bldata_product_name' : ["None"]*bldata_sample_len,
                               'bldata_keyword' : ["None"]*bldata_sample_len,
                               'cosine_similarity' : [0]*bldata_sample_len})

  buykorea_embedding=row['buykorea_embedding']
  row_count=0

  for index, row_bl in tqdm(bldata_preprocessed_openai.iterrows()):

    bldata_embedding=row_bl['bldata_embedding']

    similarity=cosine_similarity(buykorea_embedding, bldata_embedding)
    similarity_df.loc[row_count, 'bldata_product_name']=row_bl['product_name']
    similarity_df.loc[row_count, 'bldata_keyword']=row_bl['bldata_preprocessed']
    similarity_df.loc[row_count, 'cosine_similarity']=similarity

    row_count+=1

  # buykorea 제품 유사도 데이터프레임을 하나의 데이터프레임으로 병합
  openai_similarity_df=pd.concat([openai_similarity_df, similarity_df], axis=0)

display(openai_similarity_df.head())


2801it [00:04, 601.59it/s]
2801it [00:04, 600.71it/s]
2801it [00:05, 478.86it/s]
2801it [00:04, 600.46it/s]
2801it [00:04, 595.33it/s]
2801it [00:05, 503.39it/s]
2801it [00:04, 607.31it/s]
2801it [00:05, 555.86it/s]
2801it [00:05, 524.84it/s]
2801it [00:04, 613.92it/s]


,buykorea_product_name,buykorea_keyword,bldata_product_name,bldata_keyword,cosine_similarity
0,Doldori Premium Gold Essence,mask pack:skincare treatment,BODYSHOCK LOCAL REDUCER 200ML (FACE CREAM),Face Cream:Reduces local fat and tightens skin,0.452664
1,Doldori Premium Gold Essence,mask pack:skincare treatment,COSMELAN 2 (FACE CREAM),face cream:reduce pigmentation,0.430899
2,Doldori Premium Gold Essence,mask pack:skincare treatment,ANTI STRESS FACE MASK 100ML (FACE CREAM),face cream:relieve stress and soothe skin,0.600881
3,Doldori Premium Gold Essence,mask pack:skincare treatment,BLOT POWDER,Cosmetic:Oil absorption,0.277314
4,Doldori Premium Gold Essence,mask pack:skincare treatment,DERMACOLOR CAMOUFLAGE FLUID,Cosmetic:Camouflage skin imperfections,0.490007


In [ ]:
openai_similarity_df_sorted= openai_similarity_df.groupby(['buykorea_product_name'], group_keys=False, sort=False).apply(lambda x: x.sort_values('cosine_similarity', ascending=False))
openai_similarity_df_sorted.reset_index(drop=True, inplace=True)
display(openai_similarity_df_sorted.head())

,buykorea_product_name,buykorea_keyword,bldata_product_name,bldata_keyword,cosine_similarity
0,Doldori Premium Gold Essence,mask pack:skincare treatment,LITCHI REAL SOOTHING MASK PACK (BRAND-STAROVSK...,mask pack:soothing skin,0.890468
1,Doldori Premium Gold Essence,mask pack:skincare treatment,AVOCADO HIGH NUTRITION MASK PACK (BRAND-STAROV...,mask pack:nourish and hydrate skin,0.865955
2,Doldori Premium Gold Essence,mask pack:skincare treatment,LINDSAY VITAMIN MODELING MASK CUP PACK (BATCH ...,mask:moisturize and rejuvenate skin,0.826674
3,Doldori Premium Gold Essence,mask pack:skincare treatment,(LABUTE )REVIVE THE SKIN BLACK PEARL MASK,skincare mask:nourish and rejuvenate skin,0.816822
4,Doldori Premium Gold Essence,mask pack:skincare treatment,BUBBLE AWAY SHEET MASK (28G)(COSMETIC MASK),cosmetic mask:cleansing and rejuvenating skin,0.807826


In [ ]:
import pandas as pd

# 각 buykorea_product_name 별로 cosine_similarity가 높은 10개, 낮은 10개를 'Highest'로, 나머지는 'Lowest'으로 표시하는 함수
def mark_top_bottom_10(df):
    # 상위 10개와 하위 10개는 'Highest', 나머지는 'Lowest'
    df['test_row'] = ['Highest'] * 10 + ['N'] * (len(df) - 20) + ['Lowest'] * 10
    return df

# 각 buykorea_product_name별로 그룹화하여 처리
# Apply the function and then reset the index to align with the original DataFrame
openai_similarity_df_sorted['test_row'] = openai_similarity_df_sorted.groupby('buykorea_product_name', group_keys=False).apply(mark_top_bottom_10).reset_index(drop=True)['test_row']


# 결과 확인
display(openai_similarity_df_sorted)

,buykorea_product_name,buykorea_keyword,bldata_product_name,bldata_keyword,cosine_similarity,test_row
0,Doldori Premium Gold Essence,mask pack:skincare treatment,LITCHI REAL SOOTHING MASK PACK (BRAND-STAROVSK...,mask pack:soothing skin,0.890468,Highest
1,Doldori Premium Gold Essence,mask pack:skincare treatment,AVOCADO HIGH NUTRITION MASK PACK (BRAND-STAROV...,mask pack:nourish and hydrate skin,0.865955,Highest
2,Doldori Premium Gold Essence,mask pack:skincare treatment,LINDSAY VITAMIN MODELING MASK CUP PACK (BATCH ...,mask:moisturize and rejuvenate skin,0.826674,Highest
3,Doldori Premium Gold Essence,mask pack:skincare treatment,(LABUTE )REVIVE THE SKIN BLACK PEARL MASK,skincare mask:nourish and rejuvenate skin,0.816822,Highest
4,Doldori Premium Gold Essence,mask pack:skincare treatment,BUBBLE AWAY SHEET MASK (28G)(COSMETIC MASK),cosmetic mask:cleansing and rejuvenating skin,0.807826,Highest
...,...,...,...,...,...,...
28005,t:aim Barrier Sunblock 40ml,Sunblock:UV protection and after-sun care,GENFILL SOFT TOUCH 1 ML SYRINGE LOT F1-2819/1,Syringe:Administer soft touch applications,0.042460,Lowest
28006,t:aim Barrier Sunblock 40ml,Sunblock:UV protection and after-sun care,FLORAL MIST GLASS BOTTLE 400ML (REG.CERT NO.RC...,Floral Mist Glass Bottle:Dispense floral mist,0.040889,Lowest
28007,t:aim Barrier Sunblock 40ml,Sunblock:UV protection and after-sun care,GENFILL DX 1 ML SYRINGE LOT C-3719/1 (F.O.C),Syringe:Injection,0.029480,Lowest
28008,t:aim Barrier Sunblock 40ml,Sunblock:UV protection and after-sun care,PLASTIC COLOR PAINTED PALETTES SAMPLE(NAIL POL...,Nail Polish Sample Palette:Display and test na...,0.003414,Lowest


In [ ]:
openai_similarity_df_sorted.to_csv('openai_similarity_df_sorted.csv', index=False)